In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df_us22 = pd.read_excel("Data/US_22.xlsx", index_col=0)
df_us22

,Input 1,Input 2,Output 1,Output 2
DMU 1,151.42,324.96,303.04,321.64
DMU 2,234.02,516.63,235.08,248.36
DMU 3,184.48,510.16,183.22,338.86
DMU 4,208.45,476.52,251.44,299.36
DMU 5,357.47,375.87,327.81,282.08
...,...,...,...,...
DMU 996,324.76,312.23,326.97,304.73
DMU 997,296.16,284.97,317.38,301.61
DMU 998,341.29,192.10,343.57,309.25
DMU 999,264.95,316.08,352.08,339.50


In [33]:
dea_types = ["IO_VRS", "OO_CRS", "OO_VRS", "IO_CRS"]
dea_list = ["US22", "S22","US55","S55"]

for t in dea_list:
    for i, d in enumerate(dea_types):
        
        exec(f"{t}_{d} = pd.read_excel('Data/{t}_Solution.xlsx', index_col=0, header=[1], nrows=1000, skiprows = i * 1000 + i * 2)")

In [34]:
dfs = [US22_IO_VRS, US22_OO_CRS, US22_OO_VRS, US22_IO_CRS,
      S22_IO_VRS, S22_OO_CRS, S22_OO_VRS, S22_IO_CRS,
      US55_IO_VRS, US55_OO_CRS, US55_OO_VRS, US55_IO_CRS,
      S55_IO_VRS, S55_OO_CRS, S55_OO_VRS, S55_IO_CRS]

In [50]:
dfs[15]

,Efficiency
DMU,
DMU 1,0.909207
DMU 2,0.735932
DMU 3,1.000000
DMU 4,0.627744
DMU 5,0.797494
...,...
DMU 996,0.914965
DMU 997,0.856849
DMU 998,0.981252
